In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from mvn.pipeline.setup import setup_dataloaders, setup_experiment, build_env
from mvn.pipeline.training import do_train

In [3]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [5]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

cam2cam model (shared):
                      backbone has      10896 params (~ 70.2) %
                    R_backbone has       3078 params (~ 19.8) %
                    t_backbone has       1539 params (~  9.9) %
total params:                15513
cam2cam estimation => adding 15513 params to grad ...
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@17.05.2021-13:53:05
#[do_train]: epoch    0 has started!
#[cam2cam]: training batch iter 0 losses: GEO ~ 8.020, TRANS ~ 213.573, POSE ~ 76843.312, ROTO ~ 9.813, 3D ~ 594.415, SELF R ~ 47.436, SELF t ~ 22.403, TOTAL ~ 709.060
#[cam2cam]: training batch iter 0 MPJPE: ~ 431.388 mm
#[cam2cam]: training batch iter 1 losses: GEO ~ 8.113, TRANS ~ 212.625, POSE ~ 74073.781, ROTO ~ 9.364, 3D ~ 605.764, SELF R ~ 46.208, SELF t ~ 21.048, TOTAL ~ 715.117
#[cam2cam]: training batch iter 1 MPJPE: ~ 396.077 mm
#[cam2cam]: training batch iter 2 losses: GEO ~ 8.392, TRANS ~ 211.899, POSE 